In [ ]:
!onmt_translate \
  -model model_step_20000.pt \
  -src path/to/val_src.txt \
  -tgt path/to/val_tgt.txt \
  -output example_run/predictions_val.txt \
  -gpu 0 \
  -beam_size 10 \
  -n_best 5 \
  -max_length 380 \
  -batch_size 64

In [ ]:
import pandas as pd
from rdkit import Chem
from tqdm import tqdm

# Load ground truth and predictions
with open("path/to/val_tgt.txt") as f:
    targets = [line.strip() for line in f]

with open("path/to/val_src.txt") as f:
    precursors = [line.strip() for line in f]

n_best = 5
predictions = [[] for _ in range(n_best)]

with open("example_run/predictions_val.txt") as f:
    for i, line in enumerate(f):
        predictions[i % n_best].append(line.strip())

# Create evaluation DataFrame
df = pd.DataFrame({
    "target": targets,
    "precursors": precursors
})

for i in range(n_best):
    df[f"prediction_{i+1}"] = predictions[i]

# Canonicalize SMILES
def canonicalize(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return Chem.MolToSmiles(mol)
    except:
        return None

for i in range(n_best):
    df[f"canonical_prediction_{i+1}"] = df[f"prediction_{i+1}"].apply(canonicalize)

# Calculate top-n accuracy
def get_rank(row):
    for i in range(n_best):
        if row["target"] == row[f"canonical_prediction_{i+1}"]:
            return i + 1
    return None

tqdm.pandas()
df["rank"] = df.progress_apply(get_rank, axis=1)

# Report top-n accuracy
for i in range(1, n_best + 1):
    correct = df["rank"].apply(lambda x: x is not None and x <= i).sum()
    print(f"Top-{i} Accuracy: {correct / len(df) * 100:.2f}%")

# Report invalid SMILES
for i in range(n_best):
    invalid = df[f"canonical_prediction_{i+1}"].isna().sum()
    print(f"Invalid SMILES in Top-{i+1}: {invalid}")